# LSTM to Identify Stress Partens

## Importing Data

* TEMP.csv: 
Data from temperature sensor expressed degrees on the Celsius (°C) scale.

* EDA.csv:
Data from the electrodermal activity sensor expressed as microsiemens (μS).

* BVP.csv:
Data from photoplethysmograph.

* ACC.csv:
Data from 3-axis accelerometer sensor. The accelerometer is configured to measure acceleration in the range [-2g, 2g]. Therefore the unit in this file is 1/64g.
Data from x, y, and z axis are respectively in first, second, and third column.

* IBI.csv:
Time between individuals heart beats extracted from the BVP signal.
No sample rate is needed for this file.
The first column is the time (respect to the initial time) of the detected inter-beat interval expressed in seconds (s).
The second column is the duration in seconds (s) of the detected inter-beat interval (i.e., the distance in seconds from the previous beat).

* HR.csv:
Average heart rate extracted from the BVP signal.The first row is the initial time of the session expressed as unix timestamp in UTC.
The second row is the sample rate expressed in Hz.


* tags.csv:
Event mark times.
Each row corresponds to a physical button press on the device; the same time as the status LED is first illuminated.
The time is expressed as a unix timestamp in UTC and it is synchronized with initial time of the session indicated in the related data files from the corresponding session.

In [168]:
import pandas as pd
import numpy as np
import os
import pickle
from zipfile import ZipFile

In [169]:
root_path = "//Users/i546644/Documents/Unisinos/WESAD"

In [170]:
subjects_available = os.listdir(root_path)

In [171]:
subjects_available

['S5',
 'S2',
 'S3',
 'S4',
 '.DS_Store',
 'S17',
 'S10',
 'S11',
 'S16',
 'S8',
 'S6',
 'S7',
 'S9',
 'S13',
 'S14',
 'S15',
 'wesad_readme.pdf']

In [173]:
subjects_available.remove(".DS_Store")
subjects_available.remove("wesad_readme.pdf")

In [174]:
subjects_available

['S5',
 'S2',
 'S3',
 'S4',
 'S17',
 'S10',
 'S11',
 'S16',
 'S8',
 'S6',
 'S7',
 'S9',
 'S13',
 'S14',
 'S15']

In [175]:
data_names = [
    "TEMP",
    "EDA",
    "BVP",
    "ACC",
    "IBI",
    "HR",
    "TAGS"
]

In [176]:
zip_label = "_E4_Data.zip"
E4_label = "_E4_Data"

In [177]:
def unzip_files(subjects_available):
    
    subjects_data_paths = []
    
    for subject in subjects_available:
        # extracting data from zip files
        subject_E4_data_path = os.path.join(root_path, subject, subject + E4_label)
        subject_zip_data_path = os.path.join(root_path, subject, subject + zip_label)
        subjects_data_paths.append(subject_E4_data_path)
        with ZipFile(subject_zip_data_path, 'r') as zipObj:
            zipObj.extractall(subject_E4_data_path)
    
    return subjects_data_paths

In [178]:
unzip_files(subjects_available)

['//Users/i546644/Documents/Unisinos/WESAD/S5/S5_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S2/S2_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S3/S3_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S4/S4_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S17/S17_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S10/S10_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S11/S11_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S16/S16_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S8/S8_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S6/S6_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S7/S7_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S9/S9_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S13/S13_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S14/S14_E4_Data',
 '//Users/i546644/Documents/Unisinos/WESAD/S15/S15_E4_Data']

In [208]:
def get_input_chest_data(subjects_available, body_signal):
    
    chest_df_list = []
    
    for subject in subjects_available:
        
        pkl_path = os.path.join(root_path, subject, subject + ".pkl")
        f=open(pkl_path,'rb')
        data=pickle.load(f,encoding='latin1')
        
        index_df = [i for i in range(1, len(data["label"]) + 1)]
        chest_data = {
            body_signal: data["signal"]["chest"][body_signal].reshape(len(data["signal"]["chest"][body_signal]),), 
            "label": data["label"],
            "subject": data["subject"]
        }
        
        chest_df_list.append(pd.DataFrame(chest_data, index = index_df))
        
    return chest_df_list
        

In [209]:
input_chest_data = get_input_chest_data(subjects_available, "EDA")

In [210]:
len(input_chest_data)

15

In [211]:
input_chest_data[0]

,EDA,label,subject
1,3.888321,0,S5
2,3.913116,0,S5
3,3.873062,0,S5
4,3.883362,0,S5
5,3.886795,0,S5
...,...,...,...
4380596,10.231400,0,S5
4380597,10.231018,0,S5
4380598,10.231018,0,S5
4380599,10.230637,0,S5


In [212]:
input_chest_data[14]

,EDA,label,subject
1,1.362228,0,S15
2,1.357651,0,S15
3,1.345444,0,S15
4,1.353073,0,S15
5,1.344299,0,S15
...,...,...,...
3676396,4.121399,0,S15
3676397,4.118729,0,S15
3676398,4.120255,0,S15
3676399,4.122925,0,S15


### 1) Separete Data

In [213]:
train_data = input_chest_data[:9]
evaluate_data = input_chest_data[9:14]
test_data = input_chest_data[14:15]

In [214]:
y_train = []
for df in train_data:
    df.pop("subject")
    y_train.append(df.pop("label"))
X_train = train_data

In [216]:
y_evaluate = []
for df in evaluate_data:
    df.pop("subject")
    y_train.append(df.pop("label"))
X_evaluate = evaluate_data

In [217]:
y_test = []
for df in test_data:
    df.pop("subject")
    y_train.append(df.pop("label"))
X_test = test_data

## Creating Model

In [218]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [219]:
embedding_vecor_length = 32
model = Sequential()
model.add(LSTM(100))
model.add(Dense(7, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.